
# Read and preprocess imaging data from S3 and store features in SageMaker FeatureStore


In [ ]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io, os
from time import gmtime, strftime, sleep
import time
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup


## Set up SageMaker FeatureStore


In [ ]:
region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

role = get_execution_role()
s3_client = boto3.client('s3', region_name=region)

default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'sagemaker-featurestore-demo'


## Get data from S3


In [ ]:
# Get data from S3 
bucket_imag = 'guidance-multimodal-hcls-healthai-machinelearning/preprocessing'
#bucket_imag = <S3-bucket-name>

# imaging data 
data_key_imag = 'final_imaging_df.csv'
#data_key_imag = <file-name.csv>

data_location_imag = 's3://{}/{}'.format(bucket_imag, data_key_imag)
data_imaging = pd.read_csv(data_location_imag)


## Preprocess Data


In [ ]:
# Replacing NaN with zeros
data_imaging_1 = data_imaging.copy()
data_imaging_1 = data_imaging_1.replace(np.nan, 0)
data_imaging_1 = data_imaging_1.astype(str)

#Converting all diagnosis codes to a set
data_imaging_1[['alzheimers_prediction', 'coronary_heart_disease_prediction', 'stroke_prediction', 'hypertension_prediction']] = '0'
data_imaging_pred = data_imaging_1.copy()
for i in range(len(data_imaging_1)):
    data_imaging_pred.iloc[i]['diagnosisCode'] = set(data_imaging_pred.iloc[i]['diagnosisCode'].replace('\'', '').replace(' ', '').replace('{', '').replace('}', '').split(','))

# Adding a column for prediction of Alzheimer's disease code '26929004'
# Adding a column for prediction of Coronary heart disease '53741008'
# Adding a column for prediction of Stroke code '230690007'
# Adding a column for prediction of Hypertension code '59621000'
for i in range(len(data_imaging_pred)):
    if "26929004" in (data_imaging_pred.iloc[i]['diagnosisCode']):
        data_imaging_pred.iloc[i]['alzheimers_prediction']  =  '1'
    if "53741008" in (data_imaging_pred.iloc[i]['diagnosisCode']):
        data_imaging_pred.iloc[i]['coronary_heart_disease_prediction']  =  '1'
    if "230690007" in (data_imaging_pred.iloc[i]['diagnosisCode']):
        data_imaging_pred.iloc[i]['stroke_prediction']  =  '1'
    if "59621000" in (data_imaging_pred.iloc[i]['diagnosisCode']):
        data_imaging_pred.iloc[i]['hypertension_prediction']  =  '1'
print("Patients with Alzheimer's disease = ", len(data_imaging_pred[data_imaging_pred['alzheimers_prediction']=='1']))
print("Patients with Coronary Heart disease = ", len(data_imaging_pred[data_imaging_pred['coronary_heart_disease_prediction']=='1']))
print("Patients with Stroke = ", len(data_imaging_pred[data_imaging_pred['stroke_prediction']=='1']))
print("Patients with Hypertension = ", len(data_imaging_pred[data_imaging_pred['hypertension_prediction']=='1']))

# Delete columns with leakage and features irrelevant for model training
list_delete_cols = ['diagnosisDescription', 'diagnosisCode']
data_imaging_pred.drop(list_delete_cols, axis=1, inplace=True)

data_imaging_pred.head(10)


## Ingest data into FeatureStore


In [ ]:
imaging_feature_group_name = 'imaging-feature-group' 
imaging_feature_group = FeatureGroup(name=imaging_feature_group_name, sagemaker_session=feature_store_session)

current_time_sec = int(round(time.time()))

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        print (label)
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# Cast object dtype to string. SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(data_imaging_pred)

# Record identifier and event time feature names
record_identifier_feature_name = "patientID"
event_time_feature_name = "EventTime"

# Append EventTime feature
data_imaging_pred[event_time_feature_name] = pd.Series([current_time_sec]*len(data_imaging_pred), dtype="float64")

## If event time generates NaN
data_imaging_pred[event_time_feature_name] = data_imaging_pred[event_time_feature_name].fillna(0)

# Load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
imaging_feature_group.load_feature_definitions(data_frame=data_imaging_pred); # output is suppressed


def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(15)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

imaging_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)

wait_for_feature_group_creation_complete(feature_group=imaging_feature_group)

imaging_feature_group.ingest(
    data_frame=data_imaging_pred, max_workers=5, wait=True
)